In [1]:
file_name = "01TheBlackMuseumIntro.mp3"

In [2]:
# pip install pydub
# pip install SpeechRecognition
# pip install --upgrade watson-developer-cloud

INPUT_FOLDER = 'data_input/';
OUTPUT_FOLDER = 'data_output/';

In [3]:
from pydub import AudioSegment
def saveToWav(file_name):
    k = file_name.rfind("."); # find the last occurrence of dot
    audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
    audio.export(INPUT_FOLDER+file_name[:k]+'.wav', format='wav');
    return file_name[:k]+'.wav'

//anaconda/lib/python3.5/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
# IBM Git code using python api for Watson Speech-to-text
import json
from os.path import join, dirname
from watson_developer_cloud import SpeechToTextV1

# Pratyush's credential
speech_to_text = SpeechToTextV1(
    username='13f66d27-1bb3-44a7-bbf7-014f5e60c770',
    password='dKYgw0ZWjhae',
    x_watson_learning_opt_out=False
)

def getTextFromSpeech(file_name):
    with open(file_name,'rb') as audio_file:
        return speech_to_text.recognize(audio_file, content_type='audio/wav', timestamps=True,
            word_confidence=True, word_alternatives_threshold=0.01, continuous=True);

In [5]:
if '.wav' not in file_name:
    file_name = saveToWav(file_name);
result = getTextFromSpeech(INPUT_FOLDER + file_name);

FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

In [ ]:
result_dict = [];
for utterance in result["results"]:
    if "word_alternatives" not in utterance:
        raise Exception("Unknown Value Exception. No Alternatives returned")
    for hypothesis in utterance["word_alternatives"]:
        result_dict.append(hypothesis)
print(result_dict)

In [ ]:
# print(json.dumps(result_dict,indent=2))
res_start_time = []; res_end_time = []; res_confidence = []; res_word = [];
for val in result_dict:
    print(val['start_time'], '\t', val['end_time'], '\t', val['alternatives'][0]['word'], '\t', val['alternatives'][0]['confidence'])
    res_start_time.append(val['start_time']);
    res_end_time.append(val['end_time']);
    res_confidence.append(val['alternatives'][0]['confidence']);
    res_word.append(val['alternatives'][0]['word']);

In [ ]:
IGNORE_START_WORDS = 7;
HIGH_CONF_THRESHOLD = 0.9;
LOW_CONF_THRESHOLD = 0.5;
TIME_BEFORE_FIRST_WORD = 3;
TIME_AFTER_SECOND_WORD = 1;
BEEP = AudioSegment.from_file(INPUT_FOLDER+'beep.wav', 'wav');
SILENCE = AudioSegment.from_file(INPUT_FOLDER+'silence.wav', 'wav');
SILENCE = SILENCE[0:300]; # 300 ms of silence

count = 1;
i = IGNORE_START_WORDS;
while i < len(res_end_time)-1:
    # Check if the two consecutive words have high and low confidence and vice versa
    if (res_confidence[i]>HIGH_CONF_THRESHOLD and res_confidence[i-1]<LOW_CONF_THRESHOLD) or (res_confidence[i-1]>HIGH_CONF_THRESHOLD and res_confidence[i]<LOW_CONF_THRESHOLD):
        print('--', res_start_time[i-1], res_end_time[i])
        
        # Get the start and end time of the 5 seconds sample
        audio_start_time = res_start_time[i-1] - TIME_BEFORE_FIRST_WORD;
        if audio_start_time<0:
            audio_start_time = 0;
        audio_end_time = res_end_time[i] + TIME_AFTER_SECOND_WORD;
        if audio_end_time>res_end_time[len(res_end_time)-1]:
            audio_end_time = res_end_time[len(res_end_time)-1];
        
        # Make sure that the start and end time is not between a word
        for j in range(0,len(res_end_time)-1):
            if res_start_time[j+1]>audio_start_time:
                audio_start_time = res_start_time[j];
                break;
        for j in range(0,len(res_end_time)):
            if res_end_time[j]>audio_end_time:
                audio_end_time = res_end_time[j];
                i = j-1; # If i put i = j, then there won't be any overlap
                break;
        print(audio_start_time, audio_end_time)
        
        # Make an audio file with these time and store it as wav
        k = file_name.rfind("."); # find the last occurrence of dot
        audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
        audio = audio[audio_start_time*1000:audio_end_time*1000];
        audio.export(OUTPUT_FOLDER+file_name[:k]+'_'+str(count)+'_NO_B.wav', format='wav');
        
        # Make an audio file with beep and store it as wav
        audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
        audio = audio[audio_start_time*1000:res_end_time[i-2]*1000] + \
        SILENCE + BEEP + SILENCE + \
        audio[res_end_time[i-2]*1000:res_start_time[i+1]*1000] + \
        SILENCE + BEEP + SILENCE + \
        audio[res_start_time[i+1]*1000:audio_end_time*1000];
        audio.export(OUTPUT_FOLDER+file_name[:k]+'_'+str(count)+'_B.wav', format='wav');
        
        # Just store the two words
        audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
        audio = audio[res_end_time[i-2]*1000:res_start_time[i+1]*1000];
        audio.export(OUTPUT_FOLDER+file_name[:k]+'_'+str(count)+'_TWO.wav', format='wav');
        
        count = count+1;        
    i = i+1;
